# Visibility Metrics
<br>  

### References
- [Visibility Metrics and the Importance of Hiding Things](https://dzone.com/articles/visibility-metrics-and-the-importance-of-hiding-th)
- [Calculate metrics](https://101.jqassistant.org/calculate-metrics/index.html)
- [Controlling Access to Members of a Class](https://docs.oracle.com/javase/tutorial/java/javaOO/accesscontrol.html)
- [py2neo](https://py2neo.org/2021.1/)

In [ ]:
import os
import matplotlib.pyplot as plot
from py2neo import Graph

In [ ]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.
graph = Graph("bolt://localhost:7687", auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD")))

In [ ]:
def getCypherQueryFromFile(cypherFileName):
    with open(cypherFileName) as file:
        return ' '.join(file.readlines())

In [ ]:
def queryCypherToDataFrame(cypherFileName):
    return graph.run(getCypherQueryFromFile(cypherFileName)).to_data_frame()

In [ ]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [ ]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

## Artifacts

### Table 1

- List all the artifacts this notebook is based on

In [ ]:
queryCypherToDataFrame("../cypher/List_all_existing_artifacts.cypher")

## Relative Visibility Of Types

A Java class or interface may be declared with the modifier public, in which case it is visible to all classes everywhere. If a class or interface has no modifier (the default, also known as package-private), it is visible only within its own package.

The relative visibility is the number of inner components that are visible outside (public) divided by the number of all types:

$$ relative visibility = \frac{public\:types}{all\:types} $$

Using package protected types is one of many ways to improve encapsulation and implementation detail hiding.

### How to apply the results

The relative visibility is between zero (all types are package protected) and one (all types are public). A value lower than one means that there are types that are declared package protected. The lower the value is, the better implementation details are hidden. 

Non public classes can't be accessed from another package so they can be changed without affecting code in other packages. They clearly indicate functionality that only belongs to one package. This also motivates to use more classes and to split up code into smaller pieces with a single responsibility and reason to change.

### Table 1

- Show relative visibility statistics aggregated for all packages per artifact 

In [ ]:
queryCypherToDataFrame("../cypher/Visibility/Global_relative_visibility_statistics_for_types.cypher")

### Table 2

- List the top 40 packages and their artifact with the highest relative visibility

In [ ]:
queryCypherToDataFrame("../cypher/Visibility/Relative_visibility_public_types_to_all_types_per_package.cypher").head(50)